# Add chat memory to the llm model

In [7]:
import os
import sys
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_community.chat_message_histories.upstash_redis import (UpstashRedisChatMessageHistory)

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_7.ipynb')
git_add(file_path)
git_commit('Updated test 7')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

True

Login to https://upstash.com/?utm_source=Leon_LangchainPython
to save the chat history

In [ ]:
! pip install upstash_redis

In [ ]:
history = UpstashRedisChatMessageHistory(
    url = os.getenv("UPSTASH_REDIS_REST_URL"),
    token = os.getenv("UPSTASH_REDIS_REST_TOKEN"),
    session_id = "chat_1",
    ttl = 0 #conversation will never expirt. This is te time limit in seconds
)

model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0.5
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly AI assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True,
    chat_memory=history
)

chain = LLMChain(
    llm = model,
    prompt = prompt,
    memory = memory,
    verbose = True
)

if __name__ == "__main__":

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = chain.run(input=user_input)
        print("AI:", response)

